In [28]:
from pathlib import Path
import pandas as pd
from pam.core import Person
from pam.activity import Activity, Leg, Plan
from pam.scoring import CharyparNagelPlanScorer
from pam.utils import minutes_to_datetime


In [3]:
dir = Path("C:/Users/fred/Projects/caveat/logs/seq2seq/version_0")
x_path = dir / "test_input.csv"
y_path = dir / "pred.csv"

x = pd.read_csv(x_path)
y = pd.read_csv(y_path)


In [10]:
matches = 0
misses = 0
for (_, i), (_, j) in zip(x.groupby("pid"), y.groupby("pid")):
    if list(i.act) == list(j.act):
        matches += 1
    else:
        misses += 1

print(f"matches: {matches}, misses: {misses}")


matches: 127, misses: 1


In [33]:
x.head(30)

,Unnamed: 0,pid,act,start,end,mode,distance,duration
0,0,0,home,0,510,NaN,0.000000,510
1,1,0,travel,510,540,walk,0.737521,30
2,2,0,education,540,900,NaN,0.000000,360
3,3,0,travel,900,930,walk,0.737521,30
4,4,0,home,930,1440,NaN,0.000000,510
5,5,1,other,0,694,NaN,0.000000,694
6,6,1,travel,694,778,car,58.775318,84
7,7,1,other,778,1440,NaN,0.000000,662
8,8,2,home,0,390,NaN,0.000000,390
9,9,2,travel,390,400,bus,1.960758,10


In [71]:
config = {
    "default": {
        "mUM": 10,
        "utilityOfLineSwitch": -1,
        "performing": 6,
        "waiting": -0,
        "waitingPt": -1,
        "lateArrival": -18,
        "earlyDeparture": -10,
        "work": {
            "typicalDuration": "08:00:00",
            "openingTime": "06:00:00",
            "closingTime": "20:00:00",
            "latestStartTime": "09:30:00",
            "earliestEndTime": "16:00:00",
            "minimalDuration": "01:00:00",
        },
        "home": {"typicalDuration": "12:00:00", "minimalDuration": "05:00:00"},
        "depot": {"typicalDuration": "00:12:00"},
        "delivery": {"typicalDuration": "00:30:00"},
        "education": {
            "typicalDuration": "08:00:00",
            "openingTime": "08:30:00",
            "closingTime": "17:30:00",
            "latestStartTime": "09:00:00",
            "earliestEndTime": "17:00:00",
            "minimalDuration": "03:00:00",
        },
        "shop": {
            "typicalDuration": "00:30:00",
            "openingTime": "06:00:00",
            "closingTime": "20:00:00",
        },
        "other": {"typicalDuration": "00:30:00"},
        "visit": {"typicalDuration": "00:30:00"},
        "escort_other": {"typicalDuration": "00:30:00"},
        "escort_shop": {"typicalDuration": "00:30:00"},
        "escort_education": {"typicalDuration": "00:30:00"},
        "escort_work": {"typicalDuration": "00:30:00"},
        "escort_business": {"typicalDuration": "00:30:00"},
        "business": {"typicalDuration": "03:00:00"},
        "medical": {"typicalDuration": "03:00:00"},
        "car": {
            "constant": -10,
            "dailyMonetaryConstant": -1,
            "dailyUtilityConstant": -1,
            "marginalUtilityOfDistance": -0.001,
            "marginalUtilityOfTravelling": -1,
            "monetaryDistanceRate": -0.0001,
        },
        "walk": {"constant": -20},
    }
}
scorer = CharyparNagelPlanScorer(config)


def to_datetime(hours):
    return minutes_to_datetime(hours * 60)


def score_schedule(schedule):
    plan = Plan()
    for _, row in schedule.iterrows():
        start, end = to_datetime(row.start), to_datetime(row.end)
        if row.act == "travel":
            plan.add(
                Leg(
                    mode="car",
                    start_time=start,
                    end_time=end,
                    distance=row.distance/1000,
                )
            )
        else:
            plan.add(Activity(act=row.act, start_time=start, end_time=end))
    return scorer.score_plan(plan, config["default"])


In [72]:
deltas = []
for (_, i), (_, j) in zip(x.groupby("pid"), y.groupby("pid")):
    deltas.append(score_schedule(j) - score_schedule(i))


In [73]:
sum(deltas) / len(deltas)

-1140.8603059023383

In [74]:
deltas

[63.52680445961721,
 -11.000829896385198,
 -1570.4714097385788,
 -161.21839203096647,
 -109.94053090243432,
 -3195.8837054364153,
 256.68256919804037,
 1514.1467672034632,
 -10907.118279925344,
 -68.32447944225245,
 -186.56845530099423,
 -93.53554870920352,
 -91.9898640212453,
 -192.33379158101573,
 -19207.50256762041,
 2295.612486400327,
 8015.033087630858,
 10523.531280731404,
 -117.51099788872503,
 2320.2054100517744,
 -70.00279758477996,
 -164.88649791456646,
 -57.546961430839815,
 -41.00093543515396,
 15916.069179202355,
 8159.164773436592,
 12510.136136756722,
 -134.72234637627668,
 9864.072150374646,
 -1246.9159469417623,
 5462.716933631015,
 -11356.727476639704,
 -114.35783578461457,
 -20104.16777046663,
 -42821.21226217167,
 -11845.221324965925,
 -181.1602272359305,
 -14754.72315455386,
 -12984.449925475597,
 -115.00914148048403,
 -4741.834460654437,
 -217.24675867922082,
 11729.715622189271,
 -3454.541742518154,
 -113.93281016406723,
 118.31968453263107,
 -136.51537692646377,

In [75]:
x.tail(5)

,Unnamed: 0,pid,act,start,end,mode,distance,duration
851,851,127,home,0,803,NaN,0.000000,803
852,852,127,travel,803,817,car,4.322782,14
853,853,127,education,817,842,NaN,0.000000,25
854,854,127,travel,842,854,car,4.322782,12
855,855,127,home,854,1440,NaN,0.000000,586


In [76]:
y.tail(5)

,Unnamed: 0,pid,act,start,end,mode,distance,duration
851,851,127,home,0,570,NaN,473.527618,570
852,852,127,travel,570,638,car,473.687744,68
853,853,127,education,638,1056,NaN,473.779144,418
854,854,127,travel,1056,1147,car,473.661072,91
855,855,127,home,1147,1651,NaN,473.748260,504


In [82]:
print(score_schedule(x[x.pid == 127]))
print(score_schedule(y[y.pid == 127]))

8392.017206413402
-956.1119799370765
